#### lecture Feb 4, 2021
- Data cleaning with pandas, part 2
- no quiz this weekend
- no class next Thursday (Feb 11)
- midterm available next Thu and next Sunday (Feb 14)

#### Today
- merging datasets
- dataframe aggregation, you want to find the mean, median, max, min...
- kaggle to read some notebooks.

In [1]:
pwd

'E:\\DataScience\\Python'

In [2]:
import pandas as pd
import numpy as np

In [3]:
flights = pd.read_csv("flights.csv")

In [4]:
flights.info()     # 19 variables, sample size = 336776 (number of rows)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 336776 entries, 0 to 336775
Data columns (total 19 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   year            336776 non-null  int64  
 1   month           336776 non-null  int64  
 2   day             336776 non-null  int64  
 3   dep_time        328521 non-null  float64
 4   sched_dep_time  336776 non-null  int64  
 5   dep_delay       328521 non-null  float64
 6   arr_time        328063 non-null  float64
 7   sched_arr_time  336776 non-null  int64  
 8   arr_delay       327346 non-null  float64
 9   carrier         336776 non-null  object 
 10  flight          336776 non-null  int64  
 11  tailnum         334264 non-null  object 
 12  origin          336776 non-null  object 
 13  dest            336776 non-null  object 
 14  air_time        327346 non-null  float64
 15  distance        336776 non-null  int64  
 16  hour            336776 non-null  int64  
 17  minute    

(a)	Find all flights that
(i)	Had an arrival delay of two or more hours
(“arr_delay” variable is measured in minutes)
(ii)	Flew to Houston (IAH or HOU)
(“dest" is either “IAH” or “HOU”.)
(iii)	Were operated by United or Delta (You also need the dataset “airlines.csv”)

In [7]:
# a(i)
flights['flight'][flights["arr_delay"]>120]

119       4576
151       3944
218        856
268       1086
447       4497
          ... 
336579    3459
336668    4294
336724    5292
336757    3320
336763    1083
Name: flight, Length: 10034, dtype: int64

In [8]:
#  a(ii) flew to houston
flights["flight"][(flights["dest"]=="IAH")|(flights["dest"]=="HOU")]

0         1545
1         1714
32         496
81         473
89        1479
          ... 
336524     652
336527    2067
336618    1128
336694    1545
336737     475
Name: flight, Length: 9313, dtype: int64

In [9]:
# a(iii) Were operated by United or Delta
flights['flight'][(flights['carrier']=='UA')| (flights['carrier']=="DL")]

0         1545
1         1714
4          461
5         1696
12         194
          ... 
336728    1066
336737     475
336744    2363
336755     523
336762     471
Name: flight, Length: 106775, dtype: int64

(b)	How many flights have a missing 'dep_time'? What other variables are missing? What might these rows represent?

In [11]:
flights['dep_time'].isnull().sum()

8255

In [12]:
flights.isnull().any()

year              False
month             False
day               False
dep_time           True
sched_dep_time    False
dep_delay          True
arr_time           True
sched_arr_time    False
arr_delay          True
carrier           False
flight            False
tailnum            True
origin            False
dest              False
air_time           True
distance          False
hour              False
minute            False
time_hour         False
dtype: bool

(c)	Sort flights to find 
(i)	the most delayed flights.  (use “dep_delay”)
(ii)	the fastest flights. (you need to calculate “speed” = “distance”/ “air_time” )



In [14]:
flights.sort_values("dep_delay", ascending=False, inplace=True)
flights.iloc[0]   # show the information of the first row.

year                              2013
month                                1
day                                  9
dep_time                           641
sched_dep_time                     900
dep_delay                         1301
arr_time                          1242
sched_arr_time                    1530
arr_delay                         1272
carrier                             HA
flight                              51
tailnum                         N384HA
origin                             JFK
dest                               HNL
air_time                           640
distance                          4983
hour                                 9
minute                               0
time_hour         2013-01-09T14:00:00Z
Name: 7072, dtype: object

In [15]:
flights["speed"] = flights["distance"]/flights["air_time"]
flights.sort_values("speed", ascending=False, inplace=True)
flights.iloc[0]

year                              2013
month                                5
day                                 25
dep_time                          1709
sched_dep_time                    1700
dep_delay                            9
arr_time                          1923
sched_arr_time                    1937
arr_delay                          -14
carrier                             DL
flight                            1499
tailnum                         N666DN
origin                             LGA
dest                               ATL
air_time                            65
distance                           762
hour                                17
minute                               0
time_hour         2013-05-25T21:00:00Z
speed                          11.7231
Name: 216447, dtype: object

In [16]:
flights["speed"].isnull().sum()

9430

(d)	Currently dep_time is convenient to look at, but hard to compute with because they’re not really continuous numbers. Convert them to a more convenient representation of number of minutes since midnight.
(E.g. for “dep_time”, 1504 represents 15:04 (or 3:04 PM). You should convert it to 904, which means 904 minutes after midnight. Be careful, midnight can be represented by 2400, which should be converted to 0”)


In [19]:
flights["dep_time_min"] = flights["dep_time"]//100*60 + flights['dep_time']%100        
# 1705//100 = 17
# 1850//100 = 18
flights["dep_time_min"] = flights["dep_time_min"] % 1400
# 1400 is 24 hours after midnight, you have to convert it to 0
flights[["dep_time","dep_time_min"]]


,dep_time,dep_time_min
216447,1709.0,1029.0
251999,1558.0,958.0
205388,2040.0,1240.0
157516,1914.0,1154.0
10223,1559.0,959.0
...,...,...
336771,NaN,NaN
336772,NaN,NaN
336773,NaN,NaN
336774,NaN,NaN


(e)	Which carrier has the worst delays? (find the average “arr_delay” for each carrier)
(f)	Find all destinations that are flown by at least two carriers.


In [23]:
## you want to group the flights into carriers, you want to find the average arr_delay for each group.
delay = flights.groupby("carrier")["arr_delay"].mean()
delay

carrier
9E     7.379669
AA     0.364291
AS    -9.930889
B6     9.457973
DL     1.644341
EV    15.796431
F9    21.920705
FL    20.115906
HA    -6.915205
MQ    10.774733
OO    11.931034
UA     3.558011
US     2.129595
VX     1.764464
WN     9.649120
YV    15.556985
Name: arr_delay, dtype: float64

In [24]:
## 	Find all destinations that are flown by at least two carriers.
# find the number of unique carriers for each destination, 
## group all the fights into different destinatioins
flights.groupby("dest")["carrier"].nunique()

dest
ABQ    1
ACK    1
ALB    1
ANC    1
ATL    7
      ..
TPA    7
TUL    1
TVC    2
TYS    2
XNA    2
Name: carrier, Length: 105, dtype: int64

1. merging datasets
2. dataframe aggregation: mean, max, min, median
3. kaggle  notebook

(f)	group the "basic.4y", "basic.9y", and "basic.6y" categories together and call them "basic". Then do the value count for "education" again.
- recoding

In [26]:
banking = pd.read_csv("Banking_Marketing.csv")
banking["education"].value_counts()

university.degree      12170
high.school             9521
basic.9y                6045
professional.course     5244
basic.4y                4176
basic.6y                2292
unknown                 1731
illiterate                18
Basic                      2
Name: education, dtype: int64

In [29]:
education_newgroup = {"education":{"basic.9y":"basic","basic.4y":"basic",
                                   "basic.6y":"basic", "Basic":"basic"}}
banking.replace(education_newgroup, inplace=True)
banking["education"].value_counts()

basic                  12515
university.degree      12170
high.school             9521
professional.course     5244
unknown                 1731
illiterate                18
Name: education, dtype: int64

### merging datasets

In [30]:
ser1 = pd.Series(["A","B","C"], index = [1,2,3])
ser2 = pd.Series(["E","F","G"], index = [4,5,6])
pd.concat([ser1, ser2])    # one series

1    A
2    B
3    C
4    E
5    F
6    G
dtype: object

In [31]:
ser1 = pd.Series(["A","B","C"], index = [1,2,3])
ser2 = pd.Series(["E","F","G"], index = [1,2,3])
pd.concat([ser1, ser2])    # one series, use pd.concat

1    A
2    B
3    C
1    E
2    F
3    G
dtype: object

In [33]:
df1 = pd.DataFrame(np.random.randint(0,11,size=(2,3)))
df1.columns=["A", "B","C"]
df1

,A,B,C
0,3,1,2
1,0,9,5


In [34]:
df2 = pd.DataFrame(np.random.randint(0,11,size=(2,3)))
df2.columns=["B","C","D"]
df2

,B,C,D
0,10,0,5
1,5,10,7


In [36]:
pd.concat([df1,df1]) 

,A,B,C
0,3,1,2
1,0,9,5
0,3,1,2
1,0,9,5


In [41]:
## pd.merge()
df3 = pd.DataFrame({"employee":["Bob","Jake","Lisa","Sue"],
                   'group':["Accounting","Engineering","Engineering","HR"]})
df3

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [42]:
df4 = pd.DataFrame({"name":["Jake","Lisa","Sue","Bob"],
                   "hire_date":[2004,2008,2012,2014]})
df4

,name,hire_date
0,Jake,2004
1,Lisa,2008
2,Sue,2012
3,Bob,2014


In [43]:
## column for doing the merge: "employee"
pd.merge(df3,df4, left_on="employee", right_on="name")

,employee,group,name,hire_date
0,Bob,Accounting,Bob,2014
1,Jake,Engineering,Jake,2004
2,Lisa,Engineering,Lisa,2008
3,Sue,HR,Sue,2012


In [44]:
## inner merge (intersection) vs outer merge (union)
df6 = pd.DataFrame({"name":["Peter","Paul","Mary"],
                   "food":["fish","beans","bread"]})
df6

,name,food
0,Peter,fish
1,Paul,beans
2,Mary,bread


In [45]:
df7 = pd.DataFrame({"name":["Mary","Joseph"],
                   "drink":["wine","Coke"]})
df7

,name,drink
0,Mary,wine
1,Joseph,Coke


In [46]:
## inner merge (intersection)
pd.merge(df6,df7, on="name", how="inner")

,name,food,drink
0,Mary,bread,wine


In [47]:
## outer merge (union)
pd.merge(df6,df7, on="name", how="outer")

,name,food,drink
0,Peter,fish,NaN
1,Paul,beans,NaN
2,Mary,bread,wine
3,Joseph,NaN,Coke


5. Create the following 4 dataframes.
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df3 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

- merge the four dataframes

In [48]:
df1 = pd.DataFrame({'employee': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'group': ['Accounting', 'Engineering', 'Engineering', 'HR']})
df2 = pd.DataFrame({'employee': ['Lisa', 'Bob', 'Jake', 'Sue'],
                    'hire_date': [2004, 2008, 2012, 2014]})
df3 = pd.DataFrame({'group': ['Accounting', 'Engineering', 'HR'],
                    'supervisor': ['Carly', 'Guido', 'Steve']})
df4 = pd.DataFrame({'name': ['Bob', 'Jake', 'Lisa', 'Sue'],
                    'salary': [70000, 80000, 120000, 90000]})

In [49]:
df1

,employee,group
0,Bob,Accounting
1,Jake,Engineering
2,Lisa,Engineering
3,Sue,HR


In [50]:
df2

,employee,hire_date
0,Lisa,2004
1,Bob,2008
2,Jake,2012
3,Sue,2014


In [51]:
df_merged1 = pd.merge(df1,df2)
df_merged1

,employee,group,hire_date
0,Bob,Accounting,2008
1,Jake,Engineering,2012
2,Lisa,Engineering,2004
3,Sue,HR,2014


In [52]:
df3

,group,supervisor
0,Accounting,Carly
1,Engineering,Guido
2,HR,Steve


In [53]:
df_merged2 = df_merged1.merge(df3, on='group')  #  df_merged1 is the primary dataframe
df_merged2       # one to many merge

,employee,group,hire_date,supervisor
0,Bob,Accounting,2008,Carly
1,Jake,Engineering,2012,Guido
2,Lisa,Engineering,2004,Guido
3,Sue,HR,2014,Steve


In [54]:
df4

,name,salary
0,Bob,70000
1,Jake,80000
2,Lisa,120000
3,Sue,90000


In [59]:
df_final = df_merged2.merge(df4, left_on="employee", right_on="name")
df_final.drop("name", axis=1)
del df_final["name"]   # this change is permanent
df_final

,employee,group,hire_date,supervisor,salary
0,Bob,Accounting,2008,Carly,70000
1,Jake,Engineering,2012,Guido,80000
2,Lisa,Engineering,2004,Guido,120000
3,Sue,HR,2014,Steve,90000


### finally, we look at aggregation.
- max, mean, median

In [60]:
import seaborn as sns   # seaborn is for plotting graphs, seaborn consists of a lot of dataframes.
planets = sns.load_dataset("planets")

In [62]:
planets.info()    # 6 columns, sample size= 1035

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1035 entries, 0 to 1034
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   method          1035 non-null   object 
 1   number          1035 non-null   int64  
 2   orbital_period  992 non-null    float64
 3   mass            513 non-null    float64
 4   distance        808 non-null    float64
 5   year            1035 non-null   int64  
dtypes: float64(3), int64(2), object(1)
memory usage: 48.6+ KB


In [63]:
planets["method"].value_counts()

Radial Velocity                  553
Transit                          397
Imaging                           38
Microlensing                      23
Eclipse Timing Variations          9
Pulsar Timing                      5
Transit Timing Variations          4
Orbital Brightness Modulation      3
Astrometry                         2
Pulsation Timing Variations        1
Name: method, dtype: int64

In [64]:
planets.head()

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [65]:
planets["year"].max()

2014

In [66]:
planets["year"].min()

1989

In [67]:
planets["orbital_period"].mean()

2002.9175960947584

In [68]:
planets.describe()   ## we have missing values for orbital period

,number,orbital_period,mass,distance,year
count,1035.000000,992.000000,513.000000,808.000000,1035.000000
mean,1.785507,2002.917596,2.638161,264.069282,2009.070531
std,1.240976,26014.728304,3.818617,733.116493,3.972567
min,1.000000,0.090706,0.003600,1.350000,1989.000000
25%,1.000000,5.442540,0.229000,32.560000,2007.000000
50%,1.000000,39.979500,1.260000,55.250000,2010.000000
75%,2.000000,526.005000,3.040000,178.500000,2012.000000
max,7.000000,730000.000000,25.000000,8500.000000,2014.000000


In [69]:
### we want to group the planets into "method", 
## find the average value of "orbital period" for each method
planets.groupby("method")["orbital_period"].mean()

method
Astrometry                          631.180000
Eclipse Timing Variations          4751.644444
Imaging                          118247.737500
Microlensing                       3153.571429
Orbital Brightness Modulation         0.709307
Pulsar Timing                      7343.021201
Pulsation Timing Variations        1170.000000
Radial Velocity                     823.354680
Transit                              21.102073
Transit Timing Variations            79.783500
Name: orbital_period, dtype: float64

In [70]:
planets.head()     
# I want to create a column "decade", it will show the decade, when the planet was found

,method,number,orbital_period,mass,distance,year
0,Radial Velocity,1,269.300,7.10,77.40,2006
1,Radial Velocity,1,874.774,2.21,56.95,2008
2,Radial Velocity,1,763.000,2.60,19.84,2011
3,Radial Velocity,1,326.030,19.40,110.62,2007
4,Radial Velocity,1,516.220,10.50,119.47,2009


In [76]:
planets["decade"] = planets["year"] - planets["year"]%10
## planets["deace"] = planets["year"]//10 * 10
planets
## for each decade, how many planets were found by each method?
planets.groupby(["method","decade"])['number'].sum().unstack()

decade,1980,1990,2000,2010
method,,,,
Astrometry,NaN,NaN,NaN,2.0
Eclipse Timing Variations,NaN,NaN,5.0,10.0
Imaging,NaN,NaN,29.0,21.0
Microlensing,NaN,NaN,12.0,15.0
Orbital Brightness Modulation,NaN,NaN,NaN,5.0
Pulsar Timing,NaN,9.0,1.0,1.0
Pulsation Timing Variations,NaN,NaN,1.0,NaN
Radial Velocity,1.0,52.0,475.0,424.0
Transit,NaN,NaN,64.0,712.0


https://www.kaggle.com/pawanbhandarkar/covid-19-eda-man-vs-disease
https://www.kaggle.com/sayooj98/simple-eda-with-python
https://www.kaggle.com/kasrasadeghianpoor/student-performance-in-exams-data-visualization
https://www.kaggle.com/morchhlay22/nba-data
https://www.kaggle.com/rikdifos/nba-players-salary-prediction

### Assignment 4B
(a)	read in the dataset with pd.read_csv. Show the first 8 records, what’s wrong with the process of importing the dataset.

In [78]:
df = pd.read_csv("adult_income_data.csv", header=None)
df.head(8)   
# the first row in not a header, the first row is a sample
# we don't have names of the columns

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Female,0,0,40,Cuba,<=50K
5,37,Private,284582,Masters,14,Married-civ-spouse,Exec-managerial,Wife,Female,0,0,40,United-States,<=50K
6,49,Private,160187,9th,5,Married-spouse-absent,Other-service,Not-in-family,Female,0,0,16,Jamaica,<=50K
7,52,Self-emp-not-inc,209642,HS-grad,9,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,45,United-States,>50K


In [83]:
names=[]
with open("adult_income_names.txt","r") as f:      # "r", read only
    for line in f:
        f.readline()
        var = line.split(":")[0]
        names.append(var)
        
names
df.columns= names
df

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,sex,capital-gain,capital-loss,hours-per-week,native-country,wage_category
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,Female,0,0,38,United-States,<=50K
32557,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,Male,0,0,40,United-States,>50K
32558,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,Female,0,0,40,United-States,<=50K
32559,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,Male,0,0,20,United-States,<=50K


In [84]:
# (i)	group by "occupation", show the median age of each group.
df.groupby("occupation")['age'].median()

occupation
 ?                    35
 Adm-clerical         35
 Armed-Forces         29
 Craft-repair         38
 Exec-managerial      41
 Farming-fishing      39
 Handlers-cleaners    29
 Machine-op-inspct    36
 Other-service        32
 Priv-house-serv      40
 Prof-specialty       40
 Protective-serv      36
 Sales                35
 Tech-support         36
 Transport-moving     39
Name: age, dtype: int64